## OPRACOWANIE DANYCH ETAP 2

W pierwszym etapie projektu zespołowego została przygotowana wstępna wersja danych. W tym etapie dokonano wyboru okresu czasowego oraz języka badanych piosenek.
W tym etapie zostanie przygotowana analiza danych, która pozwoli na podział piosenek na kategorie. Kategorie te będą wykorzystane w kolejnym etapie do stworzenia modelu predykcyjnego.

In [34]:
#import libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

Po wczytaniu potrzebnych bibliotek, wczytujemy dane z pliku csv.

In [35]:
df = pd.read_csv('data.csv')

df.shape
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 385269 entries, 0 to 385268
Data columns (total 9 columns):
 #   Column    Non-Null Count   Dtype 
---  ------    --------------   ----- 
 0   title     385266 non-null  object
 1   tag       385269 non-null  object
 2   artist    385269 non-null  object
 3   year      385269 non-null  int64 
 4   views     385269 non-null  int64 
 5   features  385269 non-null  object
 6   lyrics    385269 non-null  object
 7   id        385269 non-null  int64 
 8   decade    385269 non-null  object
dtypes: int64(3), object(6)
memory usage: 26.5+ MB


Na początku przeprowadzone zostaną testy dla pierwszego 1000 piosenek i na podstawie wyników podjęta zostanie decyzja, jak kontynuować analizę dla pozostałych piosenek.

In [36]:
df1 = df.head(1000)

df1.head()

title  tag   artist  year   views       features   
0                Can I Live  rap    JAY-Z  1996  468624             {}  \
1         Rockin and Rollin  rap  Cam'ron  1998    6399  {"Cam\\'ron"}   
2                    DEvils  rap    JAY-Z  1996  504959             {}   
3  Its Hot Some Like It Hot  rap    JAY-Z  1999  103549             {}   
4             Its Like That  rap    JAY-Z  1998   37692  {"Kid Capri"}   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                         

Pierwszym krokiem jest przygotowanie tekstów piosenek do analizy. W tym celu zostaną usunięte znaki interpunkcyjne, a także słowa nie niosące żadnej informacji (stopwords). Następnie tekst zostanie podzielony na pojedyncze słowa (tokenizacja) i zostanie przeprowadzona lematyzacja. Lematyzacja polega na sprowadzeniu słów do ich podstawowej formy.

In [40]:
import re

def clean_text(x):
    x = x.lower()
    x = x.replace('\n', ' ')
    x = x.replace('  ', ' ')
    x = x.replace(',', '')
    x = x.replace('.', '')
    x = x.replace('?', '')
    x = x.replace('!', '')
    x = x.replace('-', '')
    x = x.replace(':', '')
    x = x.replace(';', '')
    #delete text in [] brackets
    x = re.sub(r'\[.*?\]', '', x)
    return x


df1['cleaned_text'] = df1['lyrics'].apply(lambda x: clean_text(x))


/var/folders/03/41j_jw9d7hjgv5p6jyzmkjnw0000gp/T/ipykernel_4200/2351733902.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1['cleaned_text'] = df1['lyrics'].apply(lambda x: clean_text(x))


In [43]:
#add stopwords
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()

stop = stopwords.words('english')

def tokenize(x):
    #delete stopwords
    x = ' '.join([word for word in x.split() if word not in (stop)])
    return x
 
def lemmatize(x):
    x = ' '.join([lemmatizer.lemmatize(word) for word in x.split()])
    return x

df1['cleaned_text'] = df1['cleaned_text'].apply(lambda x: tokenize(x))

#print the first 5 rows but only lyrics and cleaned_text column, but get whole text
#pd.set_option('display.max_colwidth', None)
df1[['lyrics', 'cleaned_text']].head()


[nltk_data] Error loading stopwords: <urlopen error [SSL:
[nltk_data]     CERTIFICATE_VERIFY_FAILED] certificate verify failed:
[nltk_data]     unable to get local issuer certificate (_ssl.c:992)>
/var/folders/03/41j_jw9d7hjgv5p6jyzmkjnw0000gp/T/ipykernel_4200/2229411440.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1['cleaned_text'] = df1['cleaned_text'].apply(lambda x: tokenize(x))


lyrics   
0                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                     [Produced by Irv Gotti]\n\n[Intro]\nYeah, hah, yeah, Roc-A-Fella\nWe invite you to somethin' epic, you know?\nWell, we hustle out of a sense of hopelessness\nSort of a desperation\nThrough that desperation, we become addicted\nSort of like the fiends we accustomed to servin'\nBut we feel we have nothin' to lose\nSo, we offer you, well, we offer our lives, right?\nWhat do you bring to the table?\n\n[Verse 1]\nWhile I'm watchin' every nigga watchin' me closely\nMy shit is butter for the bread, they wanna toast me\nI keep my head, both of them, where they supposed to be\nHoes'll get you sidetracked, then clapped from close feet\nI don't sleep, I'm tired, I feel wired like codeine, these days\nA brother gotta admire me from four fiends away\nMy pain, wish it was quick to see\nFrom sellin' 'caine 'til brains was fried to a fricassee\nCan't lie, at the time it never bothered me\nAt the bar, gettin' my thug on properly\nMy squad and me lack of respect for authority\nLaughin' hard, happy to be escapin' poverty, however brief\nI know this game got valleys and peaks\nExpectation for dips, for precipitation we stack chips, hardly\nThe youth I used to be, soon to see a mill'in\nNo more Big Willie, my game has grown\nPrefer you call me William\nIllin' for revenues, Rayful Edmond-like\nChannel 7 News, round seven jewels, head dead in the mic\nForgettin' all I ever knew, convenient amnesia\n"I suggest you call my lawyer, I know the procedure."\nLock my body, can't trap my mind\nEasily explain why we adapt to crime\nI'd rather die enormous than live dormant, that's how we on it\nLive at the main event, I bet a trip to Maui on it\nPresidential suites my residential for the weekend\nConfidentially speakin' in codes since I sense you peekin'\nThe NSX rental, don't be fooled, my game is mental\nWe both out of town, dog, what you tryin' to get into?\nViva Las Vegas, see ya later at the crap tables\nMeet me by the one that starts a G up\nThis way no Fraud Willies present gamblin' they re-up\nAnd we can have a pleasant time, sippin' margaritas\n[Chorus]\nGe-ge-geyeahhh\nCan I live?\nCan I live?\n\n[Verse 2]\nMy mind is infested with sick thoughts that circle\nLike a Lexus, if driven wrong it's sure to hurt you\nDual level like duplexes, in unity\nMy crew and me commit atrocities like we got immunity\nYou guessed it, manifest it\nIn tangible goods, platinum Rolex'd it\nWe don't lease, we buy the w

Aby pokazać różnicę między tekstem przed a po przeprowadzeniu powyższych operacji, policzona została różnica w ilości słów w tekście przed i po przeprowadzeniu powyższych operacji.
Jak widać, po przeprowadzeniu powyższych operacji ilość słów w tekście zmniejszyła się o około 30% w kazdym z tekstów.

In [44]:
#calculate the difference in length between the original and the cleaned text

df1['text_length'] = df1['lyrics'].apply(lambda x: len(x))
df1['cleaned_text_length'] = df1['cleaned_text'].apply(lambda x: len(x))
df1['length_difference'] = df1['text_length'] - df1['cleaned_text_length']

df1[['text_length', 'cleaned_text_length', 'length_difference']].head()

/var/folders/03/41j_jw9d7hjgv5p6jyzmkjnw0000gp/T/ipykernel_4200/1694076150.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1['text_length'] = df1['lyrics'].apply(lambda x: len(x))
/var/folders/03/41j_jw9d7hjgv5p6jyzmkjnw0000gp/T/ipykernel_4200/1694076150.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1['cleaned_text_length'] = df1['cleaned_text'].apply(lambda x: len(x))
/var/folders/03/41j_jw9d7hjgv5p6jyzmkjnw0000gp/T/ipykernel_4200/1694076150.py:5: SettingWithCopyWarning: 
A value is trying to

,text_length,cleaned_text_length,length_difference
0,2949,1970,979
1,2510,1540,970
2,3731,2490,1241
3,3409,2338,1071
4,4487,2790,1697


Próbka testowa zostanie zbadana pod kątem sentymentu występującego w piosenkach. W tym celu zostanie wykorzystana biblioteka NLTK. Dla każdej piosenki zostanie obliczony sentyment, a następnie zostanie on przypisany do jednej z kategorii: pozytywny, negatywny, neutralny.

In [46]:
import nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer

sid = SentimentIntensityAnalyzer()

df1['sentiment'] = df1['lyrics'].apply(lambda x: sid.polarity_scores(x))
df1['compound'] = df1['sentiment'].apply(lambda score_dict: score_dict['compound'])

#get the head of id, title, sentiment and compound columns
df1[['id', 'title', 'sentiment', 'compound']].head()

/var/folders/03/41j_jw9d7hjgv5p6jyzmkjnw0000gp/T/ipykernel_4200/1766940112.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1['sentiment'] = df1['lyrics'].apply(lambda x: sid.polarity_scores(x))
/var/folders/03/41j_jw9d7hjgv5p6jyzmkjnw0000gp/T/ipykernel_4200/1766940112.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1['compound'] = df1['sentiment'].apply(lambda score_dict: score_dict['compound'])


,id,title,sentiment,compound
0,3,Can I Live,"{'neg': 0.142, 'neu': 0.726, 'pos': 0.132, 'compound': -0.9409}",-0.9409
1,10,Rockin and Rollin,"{'neg': 0.069, 'neu': 0.824, 'pos': 0.107, 'compound': 0.9751}",0.9751
2,14,DEvils,"{'neg': 0.112, 'neu': 0.623, 'pos': 0.266, 'compound': 0.9991}",0.9991
3,18,Its Hot Some Like It Hot,"{'neg': 0.246, 'neu': 0.696, 'pos': 0.058, 'compound': -0.9992}",-0.9992
4,123,Its Like That,"{'neg': 0.053, 'neu': 0.836, 'pos': 0.112, 'compound': 0.9927}",0.9927


Na wykresie poniżej przedstawiono rozkład sentymentu dla próbki testowej.

Kolejnym krokiem była analiza tekstów pod kątem wsytępujacych w nim emocji. W tym celu wykorzystano bibliotekę text2emotion. Dla każdej piosenki zostały obliczone emocje, a następnie zostały one przypisane do jednej z kategorii: radość, smutek, złość, strach.

In [59]:
from text2emotion import get_emotion


df1['emotion'] = df1['lyrics'].apply(lambda x: get_emotion(x))

df1[['id', 'title', 'emotion']].head()

/var/folders/03/41j_jw9d7hjgv5p6jyzmkjnw0000gp/T/ipykernel_4200/1469079378.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1['emotion'] = df1['lyrics'].apply(lambda x: get_emotion(x))


,id,title,emotion
0,3,Can I Live,"{'Happy': 0.14, 'Angry': 0.1, 'Surprise': 0.24, 'Sad': 0.21, 'Fear': 0.32}"
1,10,Rockin and Rollin,"{'Happy': 0.08, 'Angry': 0.04, 'Surprise': 0.16, 'Sad': 0.27, 'Fear': 0.45}"
2,14,DEvils,"{'Happy': 0.11, 'Angry': 0.04, 'Surprise': 0.58, 'Sad': 0.14, 'Fear': 0.13}"
3,18,Its Hot Some Like It Hot,"{'Happy': 0.14, 'Angry': 0.06, 'Surprise': 0.31, 'Sad': 0.38, 'Fear': 0.11}"
4,123,Its Like That,"{'Happy': 0.13, 'Angry': 0.04, 'Surprise': 0.33, 'Sad': 0.23, 'Fear': 0.27}"


# CLUSTERING

To metoda polegająca na podziale zbioru danych na grupy, zwane klastrami. W obrębie jednej grupy znajdują się obiekty podobne do siebie, a różne od obiektów z innych grup. W naszym przypadku chcemy podzielić piosenki na grupy, które będą podobne do siebie pod względem sentymentu i emocji, a także podobne do siebie pod względem słów występujących w tekście.
Przetestowane zostaną dwa podejścia analiza całego tekstu piosenki oraz analiza skróconego tekstu piosenki. W przypadku analizy skróconego tekstu piosenki.

In [ ]:
from transformers import pipeline

# Create a summarization pipeline
summarizer = pipeline("summarization")

df2 = df1.head(50)
# Apply the summarizer to the 'lyrics' column
df2['summary'] = df2['lyrics'].apply(lambda x: summarizer(x, max_length=150, min_length=50, length_penalty=2.0)[0]['summary_text'])

# Display the first few rows of the DataFrame with the summaries
print(df2[['lyrics', 'summary']].head())



In [ ]:
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans

# Combine sentiment and emotion into a feature vector
features = df2[['sentiment', 'emotion']]

# Convert the sentiment and emotion dictionaries into separate columns
sentiment_columns = ['sentiment_neg', 'sentiment_neu', 'sentiment_pos', 'sentiment_compound']
emotion_columns = ['emotion_Happy', 'emotion_Angry', 'emotion_Surprise', 'emotion_Sad', 'emotion_Fear']

df2[sentiment_columns] = pd.DataFrame(df2['sentiment'].tolist(), index=df2.index)
df2[emotion_columns] = pd.DataFrame(df2['emotion'].tolist(), index=df2.index)

# Drop the original 'sentiment' and 'emotion' columns
df2.drop(['sentiment', 'emotion'], axis=1, inplace=True)

# Combine all features into a single feature vector
all_features = pd.concat([features, df2[sentiment_columns], df2[emotion_columns]], axis=1)

# Normalize the feature vector
scaler = StandardScaler()
normalized_features = scaler.fit_transform(all_features)

# Apply KMeans clustering
num_clusters = 10  # Adjust the number of clusters based on your analysis
kmeans = KMeans(n_clusters=num_clusters, random_state=42)
df1['cluster'] = kmeans.fit_predict(normalized_features)

# Display the DataFrame with cluster labels
print(df2[['lyrics', 'summary', 'sentiment', 'emotion', 'cluster']])
